# Required information

In [1]:
import os
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Load environment variables from .env
aws_account_id = os.getenv('AWS_ACCOUNT_ID')
aws_region = os.getenv('AWS_REGION')

aws_access_key = os.getenv('AWS_ACCESS_KEY')
aws_secret_key = os.getenv('AWS_SECRET_KEY')

s3_bucket_name = os.getenv('S3_BUCKET_NAME')
s3_object_key = os.getenv('S3_OBJECT_KEY')

## Specify the role name and trust policy for the Lambda service
iam_role_name = os.getenv('IAM_ROLE_NAME')

## User-dependent solution variables

In [2]:
# User setup

# Platform
ecr_image_name = "serverless-example"

# Function description
func_description='A test Lambda function'

# API
endpoint = "predict"
method_verb='POST'
stage_name = "test"

## Protocols and performance

In [4]:
# AWS setup
from os import getcwd
from deploy_scripts.utils.misc import get_lambda_usage_constraints, get_trust_policy

cwd=getcwd()

## Specify the path to the JSON file
## NOTE: Update with the actual path to your JSON file
trust_policy_file_path = cwd+"/deploy_scripts"
trust_policy = get_trust_policy(trust_policy_file_path)

## Rate limits: Harsh since this will be public facing
## Quota: Low daily limits for the same reason
usage_constraints_file_path = cwd+"/deploy_scripts"
usage_constraints = get_lambda_usage_constraints(usage_constraints_file_path)

## AWS Lambda execution role 
lambda_policy_arn = 'arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'


Loaded JSON:
{'Version': '2012-10-17', 'Statement': [{'Effect': 'Allow', 'Principal': {'Service': 'lambda.amazonaws.com'}, 'Action': 'sts:AssumeRole'}]}
Loaded JSON:
{'rate_limits': {'burstLimit': 10, 'rateLimit': 10.0}, 'quota': {'limit': 100, 'period': 'DAY'}}


# Clients

In [3]:
import boto3

# Set up the IAM client
iam_client = boto3.client('iam', region_name=aws_region)

# Set up the Lambda client
lambda_client = boto3.client('lambda', region_name=aws_region)

# Set up the API Gateway client
gateway_client = boto3.client('apigateway', region_name=aws_region)

# Development steps

1. IAM role image handling;
2. ECR image
3. Lambda function creation;
4. API Gateway

## IAM Role Image Handling

The first step: create a user on IAM with below permissions:

- **IAMUserChangePassword**: a default permission to change password 
- **IAMFullAccess**: Allows IAM management
- **AmazonEC2ContainerRegistryFullAccess**: Allows uploading image to ECR
- **AWSLambda_FullAccess**: Allows access to specific Lambda function given a role 
- **AmazonAPIGatewayAdministrator**: Allows access to specific API Gateway handling 



In [6]:
from deploy_scripts.utils.iam_utils import try_attach_role_policy

# The id "role_arn" will be used on lambda deployment
role_arn = try_attach_role_policy(iam_client, iam_role_name, trust_policy)

role_arn

Starting execution of Role policy attachment...
Finished execution of Role policy attachment.
Time taken: 0.91 seconds


'arn:aws:iam::060004687794:role/lambda-exec-role'

## ECR image

Run 2 cells below with key stroke _Shift+Enter_ to upload the docker image to ECR.

- Log in on ECR account;
- Delete existent ECR image;
- Create ECR image;
- Build docker image;
- Tag docker image;
- Push docker image to ECR.

In [7]:
from deploy_scripts.utils.ecr_utils import pipe_docker_image_to_ecr

pipe_docker_image_to_ecr(aws_account_id, aws_region, ecr_image_name)

Starting execution of Docker image upload on AWS ECR...
Logging in on ECR account...
Deleting existent ECR image...
Creating ECR image...
Building docker image...
Tagging docker image...
Pushing docker image to ECR...
Finished execution of Docker image upload on AWS ECR.
Time taken: 53.86 seconds


# Lambda function creation

Run 2 cells below with key stroke _Shift+Enter_ to generate a Lambda Function based on Docker Image

### Create function

In [9]:
from deploy_scripts.utils.ecr_utils import build_ecr_url
from deploy_scripts.utils.lambda_utils import delete_lambda_function, create_lambda_function

# Function name (not public facing)
lambda_function_name = f'lambda-fn-{ecr_image_name}'

# Retrieve (if already exists) or create a new Lambda function
routed_url = build_ecr_url(aws_account_id, aws_region, ecr_image_name)
deletion_response = delete_lambda_function(lambda_client, lambda_function_name)
create_response = create_lambda_function(lambda_client, lambda_function_name, func_description, routed_url, role_arn)

create_response

{'ResponseMetadata': {'RequestId': 'cba4c86a-905d-40b5-97a3-a5a5f4f2b1de',
  'HTTPStatusCode': 201,
  'HTTPHeaders': {'date': 'Tue, 26 Sep 2023 17:49:06 GMT',
   'content-type': 'application/json',
   'content-length': '1102',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'cba4c86a-905d-40b5-97a3-a5a5f4f2b1de'},
  'RetryAttempts': 0},
 'FunctionName': 'lambda-fn-serverless-example',
 'FunctionArn': 'arn:aws:lambda:us-east-1:060004687794:function:lambda-fn-serverless-example',
 'Role': 'arn:aws:iam::060004687794:role/lambda-exec-role',
 'CodeSize': 0,
 'Description': 'A test Lambda function',
 'Timeout': 10,
 'MemorySize': 256,
 'LastModified': '2023-09-26T17:49:06.149+0000',
 'CodeSha256': '97a2a21e7ab12b74d92b805f989fcf4bdf8feef363bd1b89c4b1a5f0d270edbc',
 'Version': '26',
 'TracingConfig': {'Mode': 'PassThrough'},
 'RevisionId': '62fbd2a6-9ce7-49ba-9ecf-de9ef8f063af',
 'State': 'Pending',
 'StateReason': 'The function is being created.',
 'StateReasonCode': 'Creating',
 'Pack

### Get function

In [18]:
from deploy_scripts.utils.lambda_utils import get_lambda_function

get_response = get_lambda_function(lambda_client, lambda_function_name)

get_response

{'ResponseMetadata': {'RequestId': '17481544-a58f-4d37-b067-e798077c3862',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 26 Sep 2023 17:49:23 GMT',
   'content-type': 'application/json',
   'content-length': '1418',
   'connection': 'keep-alive',
   'x-amzn-requestid': '17481544-a58f-4d37-b067-e798077c3862'},
  'RetryAttempts': 0},
 'Configuration': {'FunctionName': 'lambda-fn-serverless-example',
  'FunctionArn': 'arn:aws:lambda:us-east-1:060004687794:function:lambda-fn-serverless-example',
  'Role': 'arn:aws:iam::060004687794:role/lambda-exec-role',
  'CodeSize': 0,
  'Description': 'A test Lambda function',
  'Timeout': 10,
  'MemorySize': 256,
  'LastModified': '2023-09-26T17:49:06.149+0000',
  'CodeSha256': '97a2a21e7ab12b74d92b805f989fcf4bdf8feef363bd1b89c4b1a5f0d270edbc',
  'Version': '$LATEST',
  'TracingConfig': {'Mode': 'PassThrough'},
  'RevisionId': '18acf737-c5e8-4ab8-8c9b-1f19bf7e8a56',
  'State': 'Active',
  'LastUpdateStatus': 'Successful',
  'PackageType': '

### Test lambda function

In [19]:
from json import dumps, loads

# Prepare the event to pass to the Lambda function
example=[1, 2, 3, 4, 5]

# Transform into json format
payload=dumps({"body": example})

# Invoke the Lambda function
response = lambda_client.invoke(
    FunctionName=lambda_function_name,
    InvocationType='RequestResponse',
    Payload=payload
)

# Get the response from the Lambda function
result = loads(response['Payload'].read())

print(result)

{'statusCode': 200, 'headers': {'Content-Type': 'application/json'}, 'body': '[1, 4, 9, 16, 25]', 'isBase64Encoded': False}


## API Gateway setup

Run the cells below to set the Lambda Function as an Endpoint on API Gateway.

In [28]:
from deploy_scripts.utils.api_gateway_utils import deploy_rest_api, build_api_url, add_apigateway_permission
from deploy_scripts.utils.lambda_utils import get_lambda_arn, build_lambda_uri
from deploy_scripts.utils.api_gateway_utils import delete_apis_by_name, has_api

# Defines the name of the API (not public facing)
rest_api_name = lambda_function_name + '-api'

delete_apis_by_name(gateway_client, rest_api_name)

# Define the name of the API (not public facing)
rest_api_name = lambda_function_name + '-api'

# Get the Lambda function ARN
lambda_arn = get_lambda_arn(lambda_client, lambda_function_name)

# Set up integration with the Lambda function
lambda_uri = build_lambda_uri(aws_region, lambda_arn)

deployment_reponse = deploy_rest_api(\
    gateway_client,  aws_account_id, aws_region, \
    lambda_uri, rest_api_name, endpoint, method_verb, \
    usage_constraints, stage_name \
)

api_arn=deployment_reponse['arn']

try: 
    add_apigateway_permission(lambda_client, lambda_function_name, api_arn)
except lambda_client.exceptions.ResourceConflictException:
    pass

deployment_reponse

Deleted API with name 'lambda-fn-serverless-example-api' and ID 'koakf4hafh'
Starting execution of Deployment of REST API...
Finished execution of Deployment of REST API.
Time taken: 3.44 seconds


{'url': 'https://epi3e9g4w8.execute-api.us-east-1.amazonaws.com/test/predict/',
 'api_key': '7xvtVoJSjU3kP0Bb7MxOI5biL3uEnDuU5GLaraNo',
 'usage_plan_id': 'sbhk8t',
 'rest_api_id': 'epi3e9g4w8',
 'arn': 'arn:aws:execute-api:us-east-1:060004687794:epi3e9g4w8/*'}

In [30]:
api_key=deployment_reponse['api_key']
rest_api_id=deployment_reponse['rest_api_id']

# The URL by default will follow this pattern:
api_url = build_api_url(rest_api_id, aws_region, endpoint, stage_name)

print(api_key)
print(api_url)


7xvtVoJSjU3kP0Bb7MxOI5biL3uEnDuU5GLaraNo
https://epi3e9g4w8.execute-api.us-east-1.amazonaws.com/test/predict/


In [32]:
import json

deployment_info = {
    'api_key': api_key,
    'api_url': api_url,
    'method_verb': method_verb,
    # Add other information as needed
}

# Save the deployment information to a JSON file
deployment_info_filename = "deployment_info.json"

with open(deployment_info_filename, 'w') as json_file:
    json.dump(deployment_info, json_file)

print(f"Deployment information saved to {deployment_info_filename}")

Deployment information saved to deployment_info.json


### Test API Gateway endpoint

In [25]:
import json
import requests

# Prepare the event to pass to the Lambda function
example=[1, 2, 3, 4, 5]

# Transform into json format
payload=json.dumps({"body": example})

headers = {
    'Content-type': 'application/json', 
    'x-api-key': api_key,
}

resp = requests.post(api_url, headers=headers, json=example)
resp.json()


[1, 4, 9, 16, 25]

# Deployment pipeline

In [26]:
from deploy_scripts.utils.misc import get_lambda_usage_constraints, \
    get_trust_policy
from deploy_scripts.utils.iam_utils import try_attach_role_policy
from deploy_scripts.utils.ecr_utils import pipe_docker_image_to_ecr
from deploy_scripts.utils.lambda_utils import get_lambda_function_name, \
    deploy_lambda_function, \
    delete_lambda_function, \
    get_lambda_arn, \
    build_lambda_uri
from deploy_scripts.utils.api_gateway_utils import delete_apis_by_name, \
    deploy_rest_api, \
    add_apigateway_permission
from deploy_scripts.utils.misc import timing

def do_ecr_update(aws_account_id_, aws_region_, ecr_image_name_):
    # The id "role_arn" will be used on lambda deployment
    pipe_docker_image_to_ecr(aws_account_id_, aws_region_, ecr_image_name_)

def do_iam_update(iam_client_, iam_role_name_, trust_policy_file_path):
    trust_policy = get_trust_policy(trust_policy_file_path)
    
    # The id "role_arn" will be used on lambda deployment
    role_arn = try_attach_role_policy(iam_client_, iam_role_name_, trust_policy)
    
    return role_arn

def do_lambda_update(
    lambda_client_, func_description_, ecr_image_name_, \
    aws_account_id_, aws_region_, role_arn_
):
    # Gets lambda function according to ecr image name
    lambda_function_name_ = get_lambda_function_name(ecr_image_name_)

    # Deletes lambda function according to ecr image name
    delete_lambda_function(lambda_client_, lambda_function_name_)

    # Deploys lambda function of ECR image
    deploy_lambda_function(
            lambda_client_, func_description_, \
            aws_account_id_, aws_region_, \
            ecr_image_name_, role_arn_
    )
    
    # Get the Lambda function ARN
    lambda_arn = get_lambda_arn(lambda_client_, lambda_function_name_)

    # Set up integration with the Lambda function
    lambda_uri = build_lambda_uri(aws_region_, lambda_arn)
    
    return lambda_function_name_, lambda_uri
    
def do_api_update(
    gateway_client_, aws_account_id_, aws_region_, \
    lambda_uri_, lambda_function_name_, \
    endpoint_, method_verb_, stage_name_, usage_constraints_
):
    # Defines the name of the API (not public facing)
    rest_api_name_ = lambda_function_name_ + '-api'

    # Delete previous API Gateway
    # FIX: This is too harsh. Try update it! 
    delete_apis_by_name(gateway_client, rest_api_name_)

    # Deploys lambda function as API Gateway endpoint
    api_deployment_reponse = deploy_rest_api(\
        gateway_client_, aws_account_id_, aws_region_, \
        lambda_uri_, rest_api_name_,  endpoint_, method_verb_, \
        stage_name_, usage_constraints_, 
    )
    
    return api_deployment_reponse

def do_api_allowance(l_client, lambda_function_name, api_arn):
    try: 
        add_apigateway_permission(l_client, lambda_function_name, api_arn)
    except l_client.exceptions.ResourceConflictException:
        pass

@timing('Deployment of ML solution')
def deploy_solution(clients, account_info, activity_info, configuration_paths):
    # Extract metadata
    
    # Clients
    iam_client_=clients['iam']
    gateway_client_=clients['api_gateway']
    lambda_client_=clients['lambda']
    
    # AWS account information
    aws_account_id_=account_info['account_id']
    aws_region_=account_info['region']
    
    # Activity information
    iam_role_name_=activity_info['iam_role']
    ecr_image_name_=activity_info['image_name']
    func_description_=activity_info['lambda_function_description']
    endpoint_=activity_info['endpoint']
    method_verb_=activity_info['method']
    stage_name_=activity_info['stage']
    
    usage_constraints_file_path=configuration_paths['usage_constraints']
    trust_policy_file_path=configuration_paths['trust_policy']
    
    # Deployment pipeline steps: 
    
    # 1. Creates docker image and uploads to ECR
    do_ecr_update(aws_account_id_, aws_region_, ecr_image_name_)
    
    # 2. Updates iam permissions
    role_arn_=do_iam_update(iam_client_, iam_role_name_, trust_policy_file_path)
    
    # 3. Downloads respective ECR image and links to an
    lambda_function_name_, lambda_uri_=do_lambda_update(
        lambda_client_, func_description_, ecr_image_name_, \
        aws_account_id_, aws_region_, role_arn_
    )
    
    # 4. Updates API Gateway endpoint
    ## Rate limits: Harsh since this will be public facing
    ## Quota: Low daily limits for the same reason
    usage_constraints = get_lambda_usage_constraints(usage_constraints_file_path)    
    
    api_deployment_reponse=do_api_update(
        gateway_client_, aws_account_id_, aws_region_,
        lambda_uri_, lambda_function_name_, 
        endpoint_, method_verb_, stage_name_, usage_constraints
    )
    
    # 5. Allow API Gateway to access Lambda function
    api_arn=api_deployment_reponse['arn']
    do_api_allowance(lambda_client_, lambda_function_name_, api_arn)
    
    # Retrieve information from 
    rest_api_id=api_deployment_reponse['rest_api_id']
    api_key=api_deployment_reponse['api_key']
    
    # The URL by default will follow this pattern:
    api_url = build_api_url(rest_api_id, aws_region_, endpoint_, stage_name_)
    
    return {
        'api_key': api_key,
        'api_url': api_url,
        'method_verb': method_verb,
    }

In [3]:
from os import getcwd
from deploy_scripts.core import deploy_solution

cwd=getcwd()

account_info={
    "account_id": aws_account_id,
    "region": aws_region,
    "iam_role":iam_role_name
}

activity_info={
    "image_name": ecr_image_name,
    "lambda_function_description": func_description,
    "endpoint": endpoint,
    "method": method_verb,
    "stage": stage_name,
}

configuration_paths={
    "usage_constraints": cwd+"/deploy_scripts",
    "trust_policy": cwd+"/deploy_scripts"
}

deployment_info=deploy_solution(account_info, activity_info, configuration_paths)



Starting execution of Deployment of ML solution...
Starting execution of Docker image upload on AWS ECR...
Logging in on ECR account...
Deleting existent ECR image...
Creating ECR image...
Building docker image...
Tagging docker image...
Pushing docker image to ECR...
Finished execution of Docker image upload on AWS ECR.
Time taken: 43.05 seconds
Loaded JSON:
{'Version': '2012-10-17', 'Statement': [{'Effect': 'Allow', 'Principal': {'Service': 'lambda.amazonaws.com'}, 'Action': 'sts:AssumeRole'}]}
Starting execution of Role policy attachment...
Finished execution of Role policy attachment.
Time taken: 0.94 seconds
Starting execution of Lambda Function deployment...
Lambda deployment is still in progress. Waiting...
Lambda deployment is still in progress. Waiting...
Lambda deployment is still in progress. Waiting...
Lambda deployment is still in progress. Waiting...
Lambda function deployment duration: 16.06 seconds
Finished execution of Lambda Function deployment.
Time taken: 22.16 seco

In [14]:
from json import dump

# Retrieve information from deployment response
api_key=deployment_info['api_key']
api_url = deployment_info['api_url']

deployment_response = {
    'api_key': api_key,
    'api_url': api_url,
    'method_verb': method_verb,
    # Add other information as needed
}

# Save the deployment information to a JSON file
deployment_info_filename = "deployment_info.json"

with open(deployment_info_filename, 'w') as json_file:
    dump(deployment_response, json_file)

print(f"Deployment information saved to {deployment_info_filename}")

Deployment information saved to deployment_info.json


In [ ]:
{
    "api_key": "CjpUvoeCsi36hwspXzxNt8qiMrw0mYcEJ6P6j9M9", 
    "api_url": "https://85zcfk2hog.execute-api.us-east-1.amazonaws.com/test/predict/", 
    "method_verb": "POST"
}

In [31]:
from json import load

# Save the deployment information to a JSON file
deployment_info_filename = "deployment_info.json"

with open(deployment_info_filename, 'r') as json_file:
    json_payload=load(json_file)

method_verb=json_payload['method_verb']
api_key=json_payload['api_key']
api_url=json_payload['api_url']

print(f"Deployment information loaded from {deployment_info_filename}")

Deployment information loaded from deployment_info.json


In [32]:
api_url

'https://85zcfk2hog.execute-api.us-east-1.amazonaws.com/test/predict/'

In [35]:
import json
import requests

# Prepare the event to pass to the Lambda function
example=[1, 10, 3, 4, 5, 6]

# Transform into json format
payload=json.dumps({"body": example})

headers = {
    'Content-type': 'application/json', 
    'x-api-key': api_key,
}

resp = requests.post(api_url, headers=headers, json=example)

# Check the response and handle it accordingly
if resp.status_code == 200:
    response_data = resp.json()
    print(response_data)
else:
    print("Request failed with status code:", resp.status_code)
    print(resp.text)



[1, 100, 9, 16, 25, 36]
